In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Charger les données
data= pd.read_csv('/content/drive/MyDrive/Cas ebola/archive_ebola_2/ebola_2014_2016_clean.csv')



In [ ]:
data


,Country,Date,"Cumulative no. of confirmed, probable and suspected cases","Cumulative no. of confirmed, probable and suspected deaths"
0,Guinea,2014-08-29,648.0,430.0
1,Nigeria,2014-08-29,19.0,7.0
2,Sierra Leone,2014-08-29,1026.0,422.0
3,Liberia,2014-08-29,1378.0,694.0
4,Sierra Leone,2014-09-05,1261.0,491.0
...,...,...,...,...
2480,Liberia,2016-03-23,10666.0,4806.0
2481,Italy,2016-03-23,1.0,0.0
2482,Liberia,2016-03-23,5.0,4.0
2483,Nigeria,2016-03-23,20.0,8.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2485 entries, 0 to 2484
Data columns (total 4 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Country                                                     2485 non-null   object 
 1   Date                                                        2485 non-null   object 
 2   Cumulative no. of confirmed, probable and suspected cases   2477 non-null   float64
 3   Cumulative no. of confirmed, probable and suspected deaths  2485 non-null   float64
dtypes: float64(2), object(2)
memory usage: 77.8+ KB


In [ ]:

# Assurez-vous que la colonne 'Date' est au format datetime
data['Date'] = pd.to_datetime(data['Date'])

def determine_climat(row):
    country = row['Country']
    month = row['Date'].month

    if country in ['Guinea', 'Liberia', 'Mali', 'Nigeria', 'Senegal', 'Sierra Leone']:
        if 6 <= month <= 10:
            return 'Saison des pluies', 25  # Exemple: 25°C pour la saison des pluies
        else:
            return 'Saison sèche', 30  # Exemple: 30°C pour la saison sèche
    elif country in ['Italy', 'Spain', 'United Kingdom']:
        if month in [12, 1, 2]:
            return 'Hiver', 5  # Exemple: 5°C pour l'hiver
        elif 3 <= month <= 5:
            return 'Printemps', 15  # Exemple: 15°C pour le printemps
        elif 6 <= month <= 8:
            return 'Été', 25  # Exemple: 25°C pour l'été
        elif 9 <= month <= 11:
            return 'Automne', 10  # Exemple: 10°C pour l'automne
    elif country == 'United States of America':
        if month in [12, 1, 2]:
            return 'Hiver', 0  # Exemple: 0°C pour l'hiver
        elif 3 <= month <= 5:
            return 'Printemps', 15  # Exemple: 15°C pour le printemps
        elif 6 <= month <= 8:
            return 'Été', 30  # Exemple: 30°C pour l'été
        elif 9 <= month <= 11:
            return 'Automne', 10  # Exemple: 10°C pour l'automne
    return 'Non spécifié', None  # Juste au cas où

# Appliquer la fonction à chaque ligne du dataframe et séparer en deux colonnes
data[['data climat', 'température']] = data.apply(lambda row: pd.Series(determine_climat(row)), axis=1)


def get_population_density(row):
    country = row['Country']
    year = row['Date'].year

    densities = {
        'Italy': {2014: 200.18, 2015: 199.88, 2016: 199.50},
        'Guinea': {2014: 46.10, 2015: 47.29, 2016: 48.53},
        'Liberia': {2014: 40.59, 2015: 41.39, 2016: 42.29},
        'Mali': {2014: 14.15, 2015: 14.60, 2016: 15.08},
        'Nigeria': {2014: 194.18, 2015: 199.18, 2016: 204.24},
        'Senegal': {2014: 71.02, 2015: 72.98, 2016: 74.99},
        'Sierra Leone': {2014: 98.76, 2015: 101.17, 2016: 103.65},
        'Spain': {2014: 91.83, 2015: 91.77, 2016: 91.85},
        'United Kingdom': {2014: 261.6, 2015: 267.7, 2016: 269.5},
        'United States of America': {2014: 32.76, 2015: 33.02, 2016: 33.28}
    }

    # Retourne la densité de population en fonction du pays et de l'année, ou None si pas trouvé
    return densities.get(country, {}).get(year, None)

# Appliquer la fonction à chaque ligne du dataframe
data['Densité de population'] = data.apply(get_population_density, axis=1)

# Afficher les premières lignes pour vérifier le résultat
print(data.head())

        Country       Date  \
0        Guinea 2014-08-29   
1       Nigeria 2014-08-29   
2  Sierra Leone 2014-08-29   
3       Liberia 2014-08-29   
4  Sierra Leone 2014-09-05   

   Cumulative no. of confirmed, probable and suspected cases  \
0                                              648.0           
1                                               19.0           
2                                             1026.0           
3                                             1378.0           
4                                             1261.0           

   Cumulative no. of confirmed, probable and suspected deaths  \
0                                              430.0            
1                                                7.0            
2                                              422.0            
3                                              694.0            
4                                              491.0            

         data climat  température  Densité 

In [ ]:
data.isnull().sum()

,0
Country,0
Date,0
"Cumulative no. of confirmed, probable and suspected cases",8
"Cumulative no. of confirmed, probable and suspected deaths",0
data climat,0
température,0
Densité de population,0


In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

,0
Country,0
Date,0
"Cumulative no. of confirmed, probable and suspected cases",0
"Cumulative no. of confirmed, probable and suspected deaths",0
data climat,0
température,0
Densité de population,0


In [ ]:
data

,Country,Date,"Cumulative no. of confirmed, probable and suspected cases","Cumulative no. of confirmed, probable and suspected deaths",data climat,température,Densité de population
0,Guinea,2014-08-29,648.0,430.0,Saison des pluies,25,46.10
1,Nigeria,2014-08-29,19.0,7.0,Saison des pluies,25,194.18
2,Sierra Leone,2014-08-29,1026.0,422.0,Saison des pluies,25,98.76
3,Liberia,2014-08-29,1378.0,694.0,Saison des pluies,25,40.59
4,Sierra Leone,2014-09-05,1261.0,491.0,Saison des pluies,25,98.76
...,...,...,...,...,...,...,...
2480,Liberia,2016-03-23,10666.0,4806.0,Saison sèche,30,42.29
2481,Italy,2016-03-23,1.0,0.0,Printemps,15,199.50
2482,Liberia,2016-03-23,5.0,4.0,Saison sèche,30,42.29
2483,Nigeria,2016-03-23,20.0,8.0,Saison sèche,30,204.24


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Encoder la colonne 'Country' et 'data climat' pour transformer les catégories en numériques
label_encoder = LabelEncoder()
data['Country'] = label_encoder.fit_transform(data['Country'])
data['data climat'] = label_encoder.fit_transform(data['data climat'])

# Variables indépendantes (X) et dépendante (y)
X = data[['Country', 'température', 'Densité de population', 'data climat']]
y = data['Cumulative no. of confirmed, probable and suspected cases']  # La cible à prédire

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


<ipython-input-12-241ee576175a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Country'] = label_encoder.fit_transform(data['Country'])
<ipython-input-12-241ee576175a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data climat'] = label_encoder.fit_transform(data['data climat'])


Régression Linéaire

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Modèle de régression linéaire
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Prédictions
y_pred_lin = lin_reg.predict(X_test)

# Évaluation
print('Régression Linéaire - Mean Squared Error:', mean_squared_error(y_test, y_pred_lin))
print('Régression Linéaire - R2 Score:', r2_score(y_test, y_pred_lin))


Régression Linéaire - Mean Squared Error: 16162105.079139058
Régression Linéaire - R2 Score: 0.14624081624977836


 Régression Logistique

La régression logistique est plus appropriée pour des classifications binaires, mais nous allons essayer de convertir la variable cible en binaire (par exemple, les cas > 1000).

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Transformer la variable cible en binaire (par exemple, cas > 1000)
y_binary = (y > 1000).astype(int)

# Diviser les données en ensembles d'entraînement et de test pour la régression logistique
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.3, random_state=42)

# Modèle de régression logistique
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Prédictions
y_pred_log = log_reg.predict(X_test)

# Évaluation
print('Régression Logistique - Accuracy:', accuracy_score(y_test, y_pred_log))
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))


Régression Logistique - Accuracy: 0.8239247311827957
[[459  67]
 [ 64 154]]
              precision    recall  f1-score   support

           0       0.88      0.87      0.88       526
           1       0.70      0.71      0.70       218

    accuracy                           0.82       744
   macro avg       0.79      0.79      0.79       744
weighted avg       0.82      0.82      0.82       744



Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Transformer la variable cible en binaire (par exemple, cas > 1000)
y_binary = (y > 1000).astype(int)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.3, random_state=42)

# Modèle de forêt aléatoire pour classification
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Prédictions
y_pred_rf = rf_classifier.predict(X_test)

# Évaluation des performances
accuracy = accuracy_score(y_test, y_pred_rf)
conf_matrix = confusion_matrix(y_test, y_pred_rf)
class_report = classification_report(y_test, y_pred_rf)

# Résultats
print('Random Forest - Accuracy:', accuracy)
print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', class_report)


Random Forest - Accuracy: 0.9556451612903226
Confusion Matrix:
 [[493  33]
 [  0 218]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97       526
           1       0.87      1.00      0.93       218

    accuracy                           0.96       744
   macro avg       0.93      0.97      0.95       744
weighted avg       0.96      0.96      0.96       744



SVM (Support Vector Machine)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Définir les hyperparamètres à tester
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Modèle SVM
svm = SVC()

# Grid Search pour trouver les meilleurs hyperparamètres
grid = GridSearchCV(svm, param_grid, refit=True, verbose=2, cv=5)
grid.fit(X_train, y_train)

# Meilleurs paramètres trouvés
print("Best Parameters: ", grid.best_params_)

# Prédictions avec les meilleurs paramètres
y_pred_grid = grid.predict(X_test)

# Évaluation des performances
accuracy_grid = accuracy_score(y_test, y_pred_grid)
print("Accuracy after GridSearchCV: ", accuracy_grid)
print(confusion_matrix(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid))

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.3s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.3s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.3s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.6s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.6s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time= 2.5min
[CV] END ........................C=0.1, gamma=1

Réseaux Neuronaux Profonds (DNN)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Supposons que X soit vos données d'entrée et y votre cible
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Modèle de Réseau Neuronal Profond
model = Sequential()

# Ajout d'une couche d'entrée avec régularisation L2
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer='l2'))
model.add(Dropout(0.2))  # Légère réduction du dropout pour éviter trop de perte d'information

# Couches cachées supplémentaires
model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.2))

# Couche de sortie
model.add(Dense(1, activation='linear'))  # Prédiction continue (régression)

# Utilisation de l'optimiseur Adam avec un taux d'apprentissage plus bas
optimizer = Adam(learning_rate=0.0005)

# Compilation du modèle
model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

# Early stopping pour éviter le surapprentissage et réduire le taux d'apprentissage si nécessaire
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Entraînement du modèle
history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_val_scaled, y_val),
                    epochs=200,
                    batch_size=64,  # Augmenter la taille du batch
                    callbacks=[early_stopping, reduce_lr])

# Évaluation du modèle
val_mse = model.evaluate(X_val_scaled, y_val)
print(f'Validation Mean Squared Error: {val_mse}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 26836530.0000 - mse: 26836528.0000 - val_loss: 24399612.0000 - val_mse: 24399612.0000 - learning_rate: 5.0000e-04
Epoch 2/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27180390.0000 - mse: 27180390.0000 - val_loss: 24388746.0000 - val_mse: 24388746.0000 - learning_rate: 5.0000e-04
Epoch 3/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 25201354.0000 - mse: 25201352.0000 - val_loss: 24350792.0000 - val_mse: 24350792.0000 - learning_rate: 5.0000e-04
Epoch 4/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27078664.0000 - mse: 27078664.0000 - val_loss: 24233858.0000 - val_mse: 24233854.0000 - learning_rate: 5.0000e-04
Epoch 5/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 27808324.0000 - mse: 27808322.0000 - val_loss: 23947256.0000 - val_mse: 23947254.0000 - learning_rate: 5.0000e-04
Epoch 6/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 26253092.0000 - mse: 26253090.0000 - val_loss: 23352364.0000 - val_mse: 233